In [1]:
'''
Use OpenAI Gym and reinforcement learning to train a game.You may choose any game here.
'''

#import required libraries
import numpy as np
import gym
from gym import spaces


In [2]:
'''
Game Description:
Let's use a simple text-based adventure game where the player has to navigate through different rooms, collect items, and make decisions to reach a goal.
Example Game Structure:
• Rooms: "Start", "Forest", "Cave", "Treasure"
• Actions: "Go North", "Go South", "Pick Up", "Examine"
The goal is to reach the "Treasure" room.
'''


'\nGame Description:\nLet\'s use a simple text-based adventure game where the player has to navigate through different rooms, collect items, and make decisions to reach a goal.\nExample Game Structure:\n• Rooms: "Start", "Forest", "Cave", "Treasure"\n• Actions: "Go North", "Go South", "Pick Up", "Examine"\nThe goal is to reach the "Treasure" room.\n'

In [11]:
#Define the Gym-Compatible Environment
class TextAdventureEnv(gym.Env):
    def __init__(self):
        super(TextAdventureEnv, self).__init__()
        
        self.action_space = spaces.Discrete(4)  # Four actions: Go North, Go South, Pick Up, Examine
        self.observation_space = spaces.Discrete(4)  # Four rooms: Start, Forest, Cave, Treasure
        
        self.rooms = ["Start", "Forest", "Cave", "Treasure"]
        self.room_index = {room: i for i, room in enumerate(self.rooms)}
        
        # Transitions with item information
        self.transitions = {
            "Start": [("north", "Forest"), ("south", None), ("pick_up", None), ("examine", None)],
            "Forest": [("north", "Cave"), ("south", "Start"), ("pick_up", "Sword"), ("examine", None)],
            "Cave": [("north", "Treasure"), ("south", "Forest"), ("pick_up", None), ("examine", None)],
            "Treasure": [("north", None), ("south", "Cave"), ("pick_up", None), ("examine", None)]
        }
        
        self.reset()

    def reset(self):
        self.current_room = "Start"
        self.items_collected = set()
        return self.room_index[self.current_room]

    def step(self, action):
        action_map = {0: "north", 1: "south", 2: "pick_up", 3: "examine"}
        action_name = action_map[action]
        
        if action_name not in dict(self.transitions[self.current_room]):
            # Invalid action
            return self.room_index[self.current_room], -1, False, {}
        
        next_room = dict(self.transitions[self.current_room])[action_name]
        
        # Handle item picking
        if action_name == "pick_up":
            item = [item for act, item in self.transitions[self.current_room] if act == "pick_up"][0]
            if item:
                self.items_collected.add(item)
            next_room = self.current_room  # Stay in the same room after picking up an item
        
        if next_room is None:
            next_room = self.current_room
        
        self.current_room = next_room
        reward = 1 if self.current_room == "Treasure" else 0
        done = self.current_room == "Treasure"
        
        return self.room_index[self.current_room], reward, done, {}

    def render(self, mode='human'):
        print(f"You are in the {self.current_room}.")
        if self.items_collected:
            print(f"Items collected: {', '.join(self.items_collected)}")

In [12]:
#basic Q-learning agent that interacts with the Gym environment:
class QLearningAgent:
    def __init__(self, action_space, observation_space):
        self.q_table = np.zeros((observation_space.n, action_space.n))
        self.action_space = action_space
        self.learning_rate = 0.1
        self.discount = 0.9
        self.exploration_rate = 1.0
        self.exploration_decay = 0.995
        self.min_exploration_rate = 0.01

    def get_action(self, state):
        if np.random.rand() < self.exploration_rate:
            return self.action_space.sample()
        else:
            return np.argmax(self.q_table[state])

    def update_q_table(self, state, action, reward, next_state):
        best_next_action = np.max(self.q_table[next_state])
        self.q_table[state, action] += self.learning_rate * (reward + self.discount * best_next_action - self.q_table[state, action])

    def decay_exploration_rate(self):
        self.exploration_rate = max(self.min_exploration_rate, self.exploration_rate * self.exploration_decay)

In [13]:
#Training Loop
def train_agent(env, agent, episodes=1000):
    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.update_q_table(state, action, reward, next_state)
            state = next_state
        agent.decay_exploration_rate()
        if episode % 100 == 0:
            print(f"Episode {episode}: Exploration rate: {agent.exploration_rate:.2f}")

# Create the environment and agent
env = TextAdventureEnv()
agent = QLearningAgent(env.action_space, env.observation_space)

# Train the agent
train_agent(env, agent)

Episode 0: Exploration rate: 0.99
Episode 100: Exploration rate: 0.60
Episode 200: Exploration rate: 0.37
Episode 300: Exploration rate: 0.22
Episode 400: Exploration rate: 0.13
Episode 500: Exploration rate: 0.08
Episode 600: Exploration rate: 0.05
Episode 700: Exploration rate: 0.03
Episode 800: Exploration rate: 0.02
Episode 900: Exploration rate: 0.01


In [14]:
def evaluate_agent(env, agent, episodes=10):
    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            action = agent.get_action(state)  # Agent decides action based on its policy
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            state = next_state
            
        print(f"Episode {episode + 1}: Total Reward: {total_reward}")

# Evaluate the trained agent
evaluate_agent(env, agent)


Episode 1: Total Reward: 1
Episode 2: Total Reward: 1
Episode 3: Total Reward: 1
Episode 4: Total Reward: 1
Episode 5: Total Reward: 1
Episode 6: Total Reward: 1
Episode 7: Total Reward: 1
Episode 8: Total Reward: 1
Episode 9: Total Reward: 1
Episode 10: Total Reward: 1


In [15]:
def play_game(env):
    done = False
    state = env.reset()
    
    while not done:
        env.render()  # Print the current state
        
        # Get user input for action
        action = input("Enter action (north, south, pick_up, examine): ").strip().lower()
        
        # Map input to action index
        action_map = {"north": 0, "south": 1, "pick_up": 2, "examine": 3}
        if action not in action_map:
            print("Invalid action. Please choose from 'north', 'south', 'pick_up', 'examine'.")
            continue
        
        # Perform the action
        next_state, reward, done, _ = env.step(action_map[action])
        
        # Display reward and check if the game is done
        if reward > 0:
            print("Congratulations! You've found the treasure!")
        elif done:
            print("Game Over. You did not find the treasure.")
            
        # Update state
        state = next_state

# Create the environment
env = TextAdventureEnv()

# Play the game
play_game(env)


You are in the Start.


Enter action (north, south, pick_up, examine):  north


You are in the Forest.


Enter action (north, south, pick_up, examine):  south


You are in the Start.


...

You are in the Forest.


Enter action (north, south, pick_up, examine):  north


You are in the Cave.


Enter action (north, south, pick_up, examine):  north


Congratulations! You've found the treasure!


In [17]:
def print_q_table(agent, state_index_map):
    print("Q-Table:")
    for state_index in range(agent.q_table.shape[0]):
        state = state_index_map[state_index]
        print(f"State {state}: {agent.q_table[state_index]}")

# Assuming you have a state_index_map that maps indices to state descriptions
state_index_map = {i: state for i, state in enumerate(env.rooms)}
print_q_table(agent, state_index_map)


Q-Table:
State Start: [0.81       0.72893681 0.72892776 0.72883233]
State Forest: [0.9        0.72889772 0.80992096 0.8099138 ]
State Cave: [1.         0.80979338 0.89973551 0.89958087]
State Treasure: [0. 0. 0. 0.]
